In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size=300
batch_size=32
epochs=5

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/handSigns/images',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/handSigns/images',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# ✅ These were missing:
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(26, activation='softmax'))  # 26 classes


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Training the model
history=model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)




Found 6747 images belonging to 26 classes.
Found 1676 images belonging to 26 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 2739s 13s/step - accuracy: 0.7348 - loss: 1.1984 - val_accuracy: 0.9457 - val_loss: 0.1830
Epoch 2/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 195s 925ms/step - accuracy: 0.9972 - loss: 0.0114 - val_accuracy: 0.9612 - val_loss: 0.1417
Epoch 3/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 196s 931ms/step - accuracy: 0.9998 - loss: 4.4756e-04 - val_accuracy: 0.9821 - val_loss: 0.0812
Epoch 4/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 196s 930ms/step - accuracy: 0.9996 - loss: 0.0019 - val_accuracy: 0.9463 - val_loss: 0.5667
Epoch 5/5
211/211 ━━━━━━━━━━━━━━━━━━━━ 198s 939ms/step - accuracy: 0.9992 - loss: 0.0040 - val_accuracy: 0.9553 - val_loss: 0.3398


In [2]:

model.save('model.h5')



In [4]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_sign(image_path):
    img = image.load_img(image_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = list(train_generator.class_indices.keys())
    print("Predicted sign:", class_labels[class_idx])


In [5]:
predict_sign('/content/drive/MyDrive/handSigns/images/A/Image_1752190179.8612258.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted sign: A
